In [31]:
import numpy as np

# Define the unit step function
# This function returns 1 if input v >= 0, else it returns 0.
def unitStep(v):
    if v >= 0:
        return 1
    else:
        return 0

# Perceptron model that computes output based on inputs (x), weights (w), and bias (b)
# It calculates the dot product of weights and input, adds bias, then applies the unit step function.
def perceptronModel(x, w, b):
    v = np.dot(w, x) + b  # Linear combination of weights and input
    y = unitStep(v)  # Apply step function to get binary output
    return y

# Training function for perceptron with dynamic stopping
# This function updates weights and bias until either weights stop changing or no classification error is made.
def train_perceptron_dynamic(X, y, w, b, learning_rate=0.01, max_epochs=1000, tolerance=1e-4, patience=10):
    weights = w  # Initialize weights
    bias = b  # Initialize bias
    prev_weights = np.copy(weights)  # Keep track of previous weights for comparison
    patience_counter = 0  # To implement early stopping if no improvement

    for epoch in range(max_epochs):  # Loop over each epoch (training iteration)
        total_error = 0  # Reset total error for each epoch
        for i in range(len(X)):  # Loop through each input data point
            y_pred = perceptronModel(X[i], weights, bias)  # Predict the output
            error = y[i] - y_pred  # Calculate the error (difference between actual and predicted output)
            total_error += abs(error)  # Accumulate total error
            # Update weights and bias if there's an error
            weights += learning_rate * error * X[i]
            bias += learning_rate * error

        # Check weight change from previous epoch
        weight_change = np.sum(np.abs(weights - prev_weights))
        if weight_change < tolerance:  # If the change is less than the tolerance, stop training
            print(f"Training stopped due to small weight changes in epoch {epoch}")
            break
        
        # Update previous weights for next comparison
        prev_weights = np.copy(weights)

        # Early stopping based on error improvement
        if total_error == 0:  # If total error is 0, all predictions are correct, so stop training
            print(f"Training stopped due to zero classification error in epoch {epoch}")
            break

        # Patience mechanism: stop if no improvement over `patience` epochs
        if total_error > 0:  # If there's still error, increment the patience counter
            patience_counter += 1
            if patience_counter >= patience:  # If patience limit is reached, stop training
                print(f"Training stopped due to lack of improvement in epoch {epoch}")
                break
        else:
            patience_counter = 0  # Reset patience counter if there was improvement

    return weights, bias  # Return the updated weights and bias after training

# NOT logic function using a perceptron
# This function uses predefined weights and bias for the NOT gate.
def NOT_logicFunction(x, wNOT, bNOT):
    return perceptronModel(x, wNOT, bNOT)

# AND logic function using a perceptron
# This function applies weights and bias specific to the AND logic gate.
def AND_logicFunction(x, w1, bAND):
    return perceptronModel(x, w1, bAND)

# OR logic function using a perceptron
# Weights and bias specific to OR gate are used here.
def OR_logicFunction(x, w2, bOR):
    return perceptronModel(x, w2, bOR)

# XOR logic function using a combination of AND, OR, and NOT gates
# This combines basic gates to simulate the XOR function, which isn't linearly separable.
def XOR_logicFunction(x):
    # Initialize weights and biases for NOT, AND, and OR gates using random values
    wNOT = (np.random.rand())  # Random initialization for NOT gate weight
    bNOT = (np.random.rand())  # Random initialization for NOT gate bias
    x1 = np.array([0, 1])  # Input for NOT gate
    y1 = np.array([1, 0])  # Expected output for NOT gate
    wNOT, bNOT = train_perceptron_dynamic(x1, y1, wNOT, bNOT)  # Train NOT gate perceptron

    # Random initialization for AND gate weights and bias
    w1 = np.random.rand(1, 2)
    bAND = -(np.random.rand())
    y2 = np.array([0, 0, 0, 1])  # Expected output for AND gate
    w1, bAND = train_perceptron_dynamic(X, y2, w1, bAND)  # Train AND gate perceptron

    # Random initialization for OR gate weights and bias
    w2 = np.random.rand(1, 2)
    bOR = -(np.random.rand())
    y3 = np.array([0, 1, 1, 1])  # Expected output for OR gate
    w2, bOR = train_perceptron_dynamic(X, y3, w2, bOR)  # Train OR gate perceptron

    # Apply the trained AND, OR, and NOT gates to simulate XOR
    y11 = AND_logicFunction(x, w1, bAND)  # AND(x1, x2)
    y12 = OR_logicFunction(x, w2, bOR)  # OR(x1, x2)
    y13 = NOT_logicFunction(y11, wNOT, bNOT)  # NOT(AND(x1, x2))
    
    final_x = np.array([y12, y13])  # Combine outputs of OR and NOT-AND
    finalOutput = AND_logicFunction(final_x, w1, bAND)  # Apply AND to simulate XOR behavior
    return finalOutput

# Input and expected output for XOR function testing
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # Input combinations for XOR
Y = np.array([0, 1, 1, 0])


print("XOR({}, {})={}".format(X[0][0], X[0][1], XOR_logicFunction(X[0])))
print("XOR({}, {})={}".format(X[1][0], X[1][1], XOR_logicFunction(X[1])))
print("XOR({}, {})={}".format(X[2][0], X[2][1], XOR_logicFunction(X[2])))
print("XOR({}, {})={}".format(X[3][0], X[3][1], XOR_logicFunction(X[3])))


Training stopped due to lack of improvement in epoch 9
Training stopped due to lack of improvement in epoch 9
Training stopped due to small weight changes in epoch 0
XOR(0, 0)=1
Training stopped due to lack of improvement in epoch 9
Training stopped due to small weight changes in epoch 5
Training stopped due to lack of improvement in epoch 9
XOR(0, 1)=0
Training stopped due to lack of improvement in epoch 9
Training stopped due to small weight changes in epoch 5
Training stopped due to lack of improvement in epoch 9
XOR(1, 0)=0
Training stopped due to lack of improvement in epoch 9
Training stopped due to lack of improvement in epoch 9
Training stopped due to lack of improvement in epoch 9
XOR(1, 1)=1


In [9]:
import numpy as np
import tensorflow as tf

# Define the XOR input and output
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]], dtype=np.float32)

y = np.array([[0], [1], [1], [0]], dtype=np.float32)

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(40, activation='relu', input_shape=(2,)),  # Hidden layer with 2 neurons
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer
])

# Compile the model
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10000, verbose=0)

# Test the model
predictions = model.predict(X)

# Print results
for i, test in enumerate(X):
    print(f"XOR({test[0]}, {test[1]}) = {predictions[i][0]:.4f}")

for i, test in enumerate(X):
    print(f"XOR({test[0]}, {test[1]}) = {round(predictions[i][0])}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
XOR(0.0, 0.0) = 0.0272
XOR(0.0, 1.0) = 0.9890
XOR(1.0, 0.0) = 0.9890
XOR(1.0, 1.0) = 0.0079
XOR(0.0, 0.0) = 0
XOR(0.0, 1.0) = 1
XOR(1.0, 0.0) = 1
XOR(1.0, 1.0) = 0
